In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, models
import nltk
import re
import string

In [2]:
fake = 'Fake.csv'  #file with fake news data
real = 'True.csv'  #file with real news data

fake_df = pd.read_csv(fake) #create initial dfs
real_df = pd.read_csv(real) 

display(fake_df)
display(real_df)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016"
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"


,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"


In [3]:
#check for null values
null_rows_fake = fake_df[fake_df.isnull().any(axis=1)]
print("Rows with null values in Fake DataFrame:")
print(null_rows_fake)

null_rows_real = real_df[real_df.isnull().any(axis=1)]
print("Rows with null values in Real DataFrame:")
print(null_rows_real)

Rows with null values in Fake DataFrame:
Empty DataFrame
Columns: [title, text, subject, date]
Index: []
Rows with null values in Real DataFrame:
Empty DataFrame
Columns: [title, text, subject, date]
Index: []


In [4]:
#add label column
fake_df['label'] = 1
real_df['label'] = 0

In [5]:
import datetime

#function to normalize date formats
def normalize_date_format(date_str):
    #regular expressions for different date formats
    patterns = [
        (r"(\d{1,2})-(\w{3})-(\d{2})", r"\1-\2-20\3"),    # 03-Feb-16 to 03-Feb-2016
        (r"(\w{3,9}) (\d{1,2}), (\d{4})", r"\2-\1-\3"),    # February 3, 2016 to 03-February-2016
        (r"(\d{1,2}) (\w{3,9}) (\d{4})", r"\1-\2-\3")      # 03 February 2016 to 03-February-2016
    ]
    
    for pattern, replacement in patterns:
        date_str = re.sub(pattern, replacement, date_str)
    
    #convert abbreviated month names to full month names
    try:
        dt = datetime.datetime.strptime(date_str, '%d-%b-%Y')
        return dt.strftime('%d-%B-%Y')
    except ValueError:
        return date_str

#apply normalization function
fake_df['date'] = fake_df['date'].apply(normalize_date_format)
real_df['date'] = real_df['date'].apply(normalize_date_format)

fake_df['date'] = fake_df['date'].str.strip()
real_df['date'] = real_df['date'].str.strip()

display(fake_df)
display(real_df)

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,31-December-2017,1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,31-December-2017,1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,30-December-2017,1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,29-December-2017,1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,25-December-2017,1
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,16-January-2016,1
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,16-January-2016,1
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,15-January-2016,1
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,14-January-2016,1


,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,31-December-2017,0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,29-December-2017,0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,31-December-2017,0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,30-December-2017,0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,29-December-2017,0
...,...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,22-August-2017,0
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,22-August-2017,0
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,22-August-2017,0
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,22-August-2017,0


In [6]:
#combine dataframes
combined_df = pd.concat([fake_df, real_df], ignore_index=True)

#display the combined dataframe to verify the changes
display("Combined DataFrame:")
display(combined_df)

'Combined DataFrame:'

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,31-December-2017,1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,31-December-2017,1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,30-December-2017,1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,29-December-2017,1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,25-December-2017,1
...,...,...,...,...,...
44893,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,22-August-2017,0
44894,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,22-August-2017,0
44895,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,22-August-2017,0
44896,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,22-August-2017,0


In [7]:
#function to validate date formats
def validate_date_format(date_str):
    # Regular expression for the expected date format
    pattern = r"\d{1,2}-\w{3,9}-\d{4}"
    if re.match(pattern, date_str):
        return True
    else:
        return False

#identify and display rows with invalid dates
invalid_dates_df = combined_df[~combined_df['date'].apply(validate_date_format)]
print("Invalid date formats in combined_df:")
display(invalid_dates_df[['date']])  # Display rows with invalid dates

#apply validation function and drop rows with invalid dates
combined_df = combined_df[combined_df['date'].apply(validate_date_format)]

print("DataFrame after dropping rows with invalid dates:")
display(combined_df)

Invalid date formats in combined_df:


,date
9358,https://100percentfedup.com/served-roy-moore-v...
15507,https://100percentfedup.com/video-hillary-aske...
15508,https://100percentfedup.com/12-yr-old-black-co...
15839,https://fedup.wpengine.com/wp-content/uploads/...
15840,https://fedup.wpengine.com/wp-content/uploads/...
17432,https://fedup.wpengine.com/wp-content/uploads/...
17433,https://fedup.wpengine.com/wp-content/uploads/...
18933,MSNBC HOST Rudely Assumes Steel Worker Would N...
21869,https://fedup.wpengine.com/wp-content/uploads/...
21870,https://fedup.wpengine.com/wp-content/uploads/...


DataFrame after dropping rows with invalid dates:


,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,31-December-2017,1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,31-December-2017,1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,30-December-2017,1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,29-December-2017,1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,25-December-2017,1
...,...,...,...,...,...
44893,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,22-August-2017,0
44894,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,22-August-2017,0
44895,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,22-August-2017,0
44896,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,22-August-2017,0


In [8]:
duplicates = combined_df[combined_df.duplicated(keep=False)]

#drop duplicates considering all columns
combined_df = combined_df.drop_duplicates(keep='first')

print("DataFrame after dropping duplicate rows considering all columns:")
display(combined_df)

DataFrame after dropping duplicate rows considering all columns:


,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,31-December-2017,1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,31-December-2017,1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,30-December-2017,1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,29-December-2017,1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,25-December-2017,1
...,...,...,...,...,...
44893,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,22-August-2017,0
44894,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,22-August-2017,0
44895,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,22-August-2017,0
44896,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,22-August-2017,0


In [9]:
#function to parse dates with mixed formats and log unparsed dates
combined_df['date'] = pd.to_datetime(combined_df['date'], errors='coerce')

#identify and display rows with NaT dates
nat_dates_df = combined_df[combined_df['date'].isna()]
print("Rows with NaT dates in combined_df after normalization:")
print(nat_dates_df.head(50))  # Display first 50 problematic rows with all columns

Rows with NaT dates in combined_df after normalization:
Empty DataFrame
Columns: [title, text, subject, date, label]
Index: []


In [10]:
combined_df['date'] = pd.to_datetime(combined_df['date'])

#extracting temporal features
combined_df['year'] = combined_df['date'].dt.year
combined_df['day_of_year'] = combined_df['date'].dt.dayofyear

In [11]:
combined_df['content'] = combined_df['title'].astype(str) + ' ' + combined_df['text'].astype(str) + ' ' + combined_df['subject'].astype(str)

#drop unnecessary columns
combined_df_final = combined_df[['content', 'year', 'day_of_year', 'label']].copy()

display(combined_df_final)

,content,year,day_of_year,label
0,Donald Trump Sends Out Embarrassing New Year’...,2017,365,1
1,Drunk Bragging Trump Staffer Started Russian ...,2017,365,1
2,Sheriff David Clarke Becomes An Internet Joke...,2017,364,1
3,Trump Is So Obsessed He Even Has Obama’s Name...,2017,363,1
4,Pope Francis Just Called Out Donald Trump Dur...,2017,359,1
...,...,...,...,...
44893,'Fully committed' NATO backs new U.S. approach...,2017,234,0
44894,LexisNexis withdrew two products from Chinese ...,2017,234,0
44895,Minsk cultural hub becomes haven from authorit...,2017,234,0
44896,Vatican upbeat on possibility of Pope Francis ...,2017,234,0


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import nltk
from nltk.corpus import stopwords
import string
import contractions

def normalize(text):
    #initialize lemmatizer
    lemmatizer = WordNetLemmatizer()
    #convert to lowercase
    text = text.lower()
    #remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    #expand contractions
    text = contractions.fix(text)
    #remove punctuation (including apostrophes) and numbers
    text = re.sub(r"[^\w\s]", '', text)
    #remove numbers
    text = re.sub(r'\d+', '', text)
    #remove extra whitespace
    text = ' '.join(text.split())
    #remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word not in stop_words)
    #lemmatize the text
    text = ' '.join(lemmatizer.lemmatize(word, pos='v') for word in text.split())
    
    return text

#apply function
combined_df_final['content'] = combined_df_final['content'].apply(normalize)

In [13]:
display(combined_df_final)
print(combined_df_final['label'].value_counts())

,content,year,day_of_year,label
0,donald trump send embarrass new years eve mess...,2017,365,1
1,drink brag trump staffer start russian collusi...,2017,365,1
2,sheriff david clarke become internet joke thre...,2017,364,1
3,trump obsess even obamas name cod website imag...,2017,363,1
4,pope francis call donald trump christmas speec...,2017,359,1
...,...,...,...,...
44893,fully commit nato back new yous approach afgha...,2017,234,0
44894,lexisnexis withdraw two products chinese marke...,2017,234,0
44895,minsk cultural hub become authorities minsk re...,2017,234,0
44896,vatican upbeat possibility pope francis visit ...,2017,234,0


label
1    23468
0    21211
Name: count, dtype: int64


In [14]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler

#split data into training and testing sets
train_df, test_df = train_test_split(combined_df_final, test_size=0.2, random_state=42)

train_content = train_df['content'].tolist()
test_content = test_df['content'].tolist()

#initialize tokenizer and convert texts to sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_content)

#define vocab size
vocab_size = len(tokenizer.word_index) + 1  # Define vocabulary size
print(f"Vocabulary size: {vocab_size}")

#convert texts to sequences
train_sequences = tokenizer.texts_to_sequences(train_content)
test_sequences = tokenizer.texts_to_sequences(test_content)

#max length
sequence_lengths = [len(seq) for seq in train_sequences + test_sequences]
max_length = int(np.percentile(sequence_lengths, 95))
print(f"Max sequence length: {max_length}")

embedding_dim = 100  # Embedding dimension to be used
print(f"Embedding dim: {embedding_dim}")

#pad sequences
X_train_text = pad_sequences(train_sequences, maxlen=max_length)
X_test_text = pad_sequences(test_sequences, maxlen=max_length)

#extract date elements
train_dates = train_df[['year', 'day_of_year']].values
test_dates = test_df[['year', 'day_of_year']].values

#standardize date elements
scaler = StandardScaler()
train_dates = scaler.fit_transform(train_dates)
test_dates = scaler.transform(test_dates)

#combine text sequences with date elements
X_train = np.hstack((X_train_text, train_dates))
X_test = np.hstack((X_test_text, test_dates))

#define labels
y_train = np.array(train_df['label'])
y_test = np.array(test_df['label'])

#print shapes of training and testing data (used for earlier debug)
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

Vocabulary size: 178629
Max sequence length: 528
Embedding dim: 100
X_train shape: (35743, 530)
y_train shape: (35743,)
X_test shape: (8936, 530)
y_test shape: (8936,)


In [15]:
from numpy import zeros
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Embedding, Dropout, Conv1D, MaxPooling1D, BatchNormalization, Input, concatenate
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

#initialize model
model = Sequential()

#define the model with multiple inputs
input_text = Input(shape=(max_length,), name='text_input')
embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length)(input_text)
conv1d = Conv1D(filters=64, kernel_size=5, activation='relu')(embedding)
maxpool = MaxPooling1D(pool_size=2)(conv1d)
dropout = Dropout(0.3)(maxpool)
flatten = Flatten()(dropout)

input_dates = Input(shape=(train_dates.shape[1],), name='date_input')

#combine text and date features
combined = concatenate([flatten, input_dates])

dense = Dense(64, activation='relu')(combined)
output = Dense(1, activation='sigmoid')(dense)

model = Model(inputs=[input_text, input_dates], outputs=output)

#compile the network
optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

#print model summary
model.summary()

/Users/raylesser/anaconda3/envs/tf_macos_clean/lib/python3.9/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2024-08-08 22:38:37.529509: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-08-08 22:38:37.529540: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-08-08 22:38:37.529550: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-08-08 22:38:37.529573: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-08 22:38:37.529594: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ text_input          │ (None, 528)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 528, 100)  │ 17,862,900 │ text_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 524, 64)   │     32,064 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d       │ (None, 262, 64)   │          0 │ conv1d[0][0]      │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 262, 64)   │          0 │ max_pooling1d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 16768)     │          0 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ date_input          │ (None, 2)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 16770)     │          0 │ flatten[0][0],    │
│ (Concatenate)       │                   │            │ date_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │  1,073,344 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │         65 │ dense[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 18,968,373 (72.36 MB)

 Trainable params: 18,968,373 (72.36 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
from sklearn.metrics import classification_report, accuracy_score

#define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss')

#fit the network with validation data for early stopping and checkpoints
history = model.fit([X_train_text, train_dates], y_train, epochs=10, batch_size=32, verbose=2, validation_split=0.2, callbacks=[early_stopping, model_checkpoint])

#evaluate on the training set
train_loss, train_acc = model.evaluate([X_train_text, train_dates], y_train, verbose=0)
print(f"Training loss: {train_loss}, Training accuracy: {train_acc}")

#evaluate on the test set
test_loss, test_acc = model.evaluate([X_test_text, test_dates], y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_acc}")

Epoch 1/10


2024-08-08 22:38:38.136348: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


894/894 - 115s - 129ms/step - accuracy: 0.9858 - loss: 0.0310 - val_accuracy: 1.0000 - val_loss: 8.1479e-06
Epoch 2/10
894/894 - 110s - 123ms/step - accuracy: 1.0000 - loss: 8.0870e-06 - val_accuracy: 1.0000 - val_loss: 3.4997e-06
Epoch 3/10
894/894 - 105s - 117ms/step - accuracy: 1.0000 - loss: 3.9817e-06 - val_accuracy: 1.0000 - val_loss: 1.5030e-06
Epoch 4/10
894/894 - 102s - 114ms/step - accuracy: 1.0000 - loss: 2.2146e-06 - val_accuracy: 1.0000 - val_loss: 9.0039e-07
Epoch 5/10
894/894 - 98s - 109ms/step - accuracy: 1.0000 - loss: 1.0520e-06 - val_accuracy: 1.0000 - val_loss: 4.6546e-07
Epoch 6/10
894/894 - 93s - 104ms/step - accuracy: 1.0000 - loss: 5.5261e-07 - val_accuracy: 1.0000 - val_loss: 2.7659e-07
Epoch 7/10
894/894 - 90s - 101ms/step - accuracy: 1.0000 - loss: 5.0232e-07 - val_accuracy: 1.0000 - val_loss: 1.0450e-07
Epoch 8/10
894/894 - 85s - 95ms/step - accuracy: 1.0000 - loss: 1.7128e-07 - val_accuracy: 1.0000 - val_loss: 8.3808e-08
Epoch 9/10
894/894 - 82s - 92ms/step

In [17]:
from sklearn.metrics import classification_report

#make predictions on the training set
predictions_train = model.predict([X_train_text, train_dates])
y_pred_train_classes = (predictions_train > 0.5).astype("int32")

#print classification report for the training set
print("Classification report for the training set:")
print(classification_report(y_train, y_pred_train_classes))

#make predictions on the test set
predictions_test = model.predict([X_test_text, test_dates])
y_pred_test_classes = (predictions_test > 0.5).astype("int32")

#print classification report for the test set
print("Classification report for the test set:")
print(classification_report(y_test, y_pred_test_classes))

1117/1117 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step
Classification report for the training set:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     16986
           1       1.00      1.00      1.00     18757

    accuracy                           1.00     35743
   macro avg       1.00      1.00      1.00     35743
weighted avg       1.00      1.00      1.00     35743

280/280 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
Classification report for the test set:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4225
           1       1.00      1.00      1.00      4711

    accuracy                           1.00      8936
   macro avg       1.00      1.00      1.00      8936
weighted avg       1.00      1.00      1.00      8936



In [ ]:
import keras_tuner as kt

#define model with hyperparameters
def build_model(hp):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, 
                        output_dim=hp.Int('embedding_dim', min_value=50, max_value=300, step=50), 
                        input_length=max_length))
    model.add(Conv1D(filters=hp.Int('filters', min_value=32, max_value=128, step=32), 
                     kernel_size=hp.Int('kernel_size', min_value=3, max_value=7, step=2), 
                     activation='relu'))
    model.add(MaxPooling1D(pool_size=hp.Int('pool_size', min_value=2, max_value=4, step=1)))
    model.add(Dropout(hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(Flatten())
    model.add(Dense(hp.Int('dense_units', min_value=32, max_value=128, step=32), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')), 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])
    
    return model

#initialize the tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Number of hyperparameter combinations to try
    executions_per_trial=1,
    directory='my_dir',  # Directory to save results
    project_name='my_project'
)

#print the search space summary
tuner.search_space_summary()

#set up callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss')

#perform hyperparameter tuning
tuner.search(X_train, y_train, 
             epochs=10, 
             batch_size=32, 
             validation_split=0.2, 
             callbacks=[early_stopping, model_checkpoint])

# Print the best hyperparameters
tuner.results_summary()

In [ ]:
# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Predict on the test set
y_pred = (best_model.predict(X_test) > 0.5).astype(int).flatten()

# Generate and print classification report
report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)